<a href="https://colab.research.google.com/github/dvguruprasad/colab-wip/blob/main/lda_async_interviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv 

all_data=[]
with open('/content/query_result_2022-08-24T07_22_23.39294Z.csv', mode='r') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    all_data.append(row)
  all_data = all_data[1:]

all = [i['data'] for i in all_data]
not_recommended = [i['data'] for i in all_data if i['recommendation'] == 'Not Recommended']
recommended = [i['data'] for i in all_data if i['recommendation'] == 'Recommended']

corpus = all

In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


stemmer = PorterStemmer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [11]:
doc_sample = corpus[0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

processed_docs = [preprocess(doc) for doc in corpus]
print(processed_docs[0])

original document: 
['', 'with', 'warehouse', 'management', 'systems', 'such', 'as', "let's", 'see', 'the', 'RF', 'scanners', "I've", 'used', 'them', 'for', 'over', 'a', 'year', "I've", 'had', 'plenty', 'of', 'experience', "I've", 'been', 'trained', 'to', 'do', 'most', 'of', 'the', 'functions', 'on', 'them', 'other', 'than', 'like', 'some', 'of', 'the', 'more', 'complicated', 'ones', 'most', 'of', 'the', 'functions', 'our', 'church', 'I', 'was', 'trained', 'on', 'with', 'RF', 'scanner', 'were', 'moving', '', 'from', 'one', 'slide', 'to', 'another', 'and', 'or', 'cakey', 'in', 'receiving', 'items', 'and', 'moving', 'them', 'to', 'pallet', 'slots', 'or', 'inputting', 'from', 'a', 'palette', 'slot', 'to', 'a', 'pixel', 'lat', 'my', 'Initiative', 'for', 'learning', 'something', 'new', 'was', 'the', 'simple', 'fact', 'of', '', 'anything', 'I', 'try', 'to', 'learn', 'something', 'new', 'to', 'me', 'I', 'look', 'at', 'it', 'and', 'think', 'about', 'what', 'I', 'can', 'use', 'it', 'for', 'in',

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 action
1 appli
2 applic
3 appropri
4 arriv
5 ask
6 await
7 balanc
8 base
9 best
10 cakey


In [20]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(len(bow_corpus))

bow_doc_10 = bow_corpus[10]
for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], 
                                               dictionary[bow_doc_10[i][0]], bow_doc_10[i][1]))

927
Word 5 ("ask") appears 2 time.
Word 8 ("career") appears 2 time.
Word 10 ("certain") appears 1 time.
Word 27 ("field") appears 2 time.
Word 30 ("follow") appears 1 time.
Word 31 ("friend") appears 1 time.
Word 32 ("function") appears 1 time.
Word 34 ("get") appears 3 time.
Word 35 ("give") appears 2 time.
Word 39 ("have") appears 1 time.
Word 40 ("home") appears 2 time.
Word 43 ("initi") appears 4 time.
Word 46 ("knowledg") appears 2 time.
Word 47 ("later") appears 3 time.
Word 48 ("life") appears 1 time.
Word 52 ("mean") appears 1 time.
Word 56 ("order") appears 1 time.
Word 67 ("school") appears 1 time.
Word 68 ("servic") appears 1 time.
Word 75 ("team") appears 6 time.
Word 79 ("type") appears 1 time.
Word 80 ("understand") appears 5 time.
Word 86 ("activ") appears 1 time.
Word 89 ("articl") appears 1 time.
Word 93 ("build") appears 3 time.
Word 103 ("earli") appears 1 time.
Word 106 ("enjoy") appears 1 time.
Word 112 ("gain") appears 1 time.
Word 121 ("morn") appears 2 time.
Wo

In [22]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.09228620446161498),
 (1, 0.06312257899318931),
 (2, 0.09156118576545902),
 (3, 0.10382662869668012),
 (4, 0.11573180004869624),
 (5, 0.02388169575330429),
 (6, 0.11573180004869624),
 (7, 0.050857136061968185),
 (8, 0.059066241175054775),
 (9, 0.23684540327716644),
 (10, 0.042770554141443894),
 (11, 0.104934751350226),
 (12, 0.18457240892322996),
 (13, 0.05592535090437536),
 (14, 0.3424328729578433),
 (15, 0.050324884049593524),
 (16, 0.0542707292059491),
 (17, 0.08882788660932817),
 (18, 0.05176624166411207),
 (19, 0.1085414584118982),
 (20, 0.3379146879400812),
 (21, 0.0420986308762586),
 (22, 0.08458272649848059),
 (23, 0.0705016142593262),
 (24, 0.0476385074209489),
 (25, 0.06719906210564801),
 (26, 0.0420986308762586),
 (27, 0.060274714840865724),
 (28, 0.041966071891168605),
 (29, 0.06981071731637198),
 (30, 0.055086386248110235),
 (31, 0.035786867105320405),
 (32, 0.30219917388403283),
 (33, 0.059066241175054775),
 (34, 0.021219923466469295),
 (35, 0.03023777917843186),
 (

In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"phone" + 0.009*"task" + 0.009*"mean" + 0.008*"ask" + 0.008*"take" + 0.008*"speak" + 0.008*"understand" + 0.007*"actual" + 0.007*"yeah" + 0.007*"pretti"
Topic: 1 
Words: 0.014*"probabl" + 0.012*"phone" + 0.012*"actual" + 0.011*"differ" + 0.010*"task" + 0.008*"yeah" + 0.008*"goal" + 0.008*"comput" + 0.007*"play" + 0.007*"get"
Topic: 2 
Words: 0.010*"store" + 0.010*"understand" + 0.009*"differ" + 0.009*"servic" + 0.008*"great" + 0.008*"team" + 0.008*"basic" + 0.008*"take" + 0.007*"situat" + 0.007*"pretti"
Topic: 3 
Words: 0.013*"actual" + 0.013*"yeah" + 0.012*"team" + 0.011*"definit" + 0.009*"goal" + 0.008*"phone" + 0.008*"servic" + 0.008*"get" + 0.008*"speak" + 0.007*"differ"
Topic: 4 
Words: 0.015*"actual" + 0.011*"yeah" + 0.010*"understand" + 0.009*"make" + 0.009*"phone" + 0.009*"issu" + 0.008*"enjoy" + 0.007*"differ" + 0.007*"problem" + 0.007*"usual"
Topic: 5 
Words: 0.011*"pretti" + 0.011*"yeah" + 0.010*"store" + 0.008*"ask" + 0.008*"stay" + 0.008*"comput" + 0

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"languag" + 0.004*"speak" + 0.004*"translat" + 0.004*"actual" + 0.003*"probabl" + 0.003*"team" + 0.003*"yeah" + 0.003*"problem" + 0.003*"build" + 0.003*"phone"
Topic: 1 Word: 0.005*"consol" + 0.004*"basic" + 0.003*"order" + 0.003*"problem" + 0.003*"pick" + 0.003*"internet" + 0.003*"pretti" + 0.003*"electron" + 0.003*"credit" + 0.003*"phone"
Topic: 2 Word: 0.004*"yeah" + 0.004*"iphon" + 0.004*"phone" + 0.003*"target" + 0.003*"okay" + 0.003*"take" + 0.003*"video" + 0.003*"actual" + 0.003*"forklift" + 0.003*"depot"
Topic: 3 Word: 0.004*"goal" + 0.004*"task" + 0.003*"store" + 0.003*"truck" + 0.003*"listen" + 0.003*"complet" + 0.003*"electron" + 0.003*"enjoy" + 0.003*"believ" + 0.003*"import"
Topic: 4 Word: 0.004*"enjoy" + 0.004*"understand" + 0.004*"differ" + 0.003*"mean" + 0.003*"team" + 0.003*"actual" + 0.003*"posit" + 0.003*"interest" + 0.003*"everybodi" + 0.003*"issu"
Topic: 5 Word: 0.004*"everybodi" + 0.003*"actual" + 0.003*"yeah" + 0.003*"comput" + 0.003*"task" +